타이타닉 데이터셋을 가지고 머신러닝이 아닌 딥러닝을 적용해보았습니다.

### 1. Data loading & Preprocessing

**필요한 라이브러리를 Import 합니다.**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**데이터를 로드합니다.**

In [2]:
titanic_df = pd.read_csv("titanic.csv")
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**데이터를 딥러닝 돌리기 위해 다듬어 줍니다.**

In [3]:
del titanic_df['Cabin'] # 너무 많은 결측치가 존재하기 때문에 제거합니다.
del titanic_df['PassengerId'] # Passenger 번호는 큰 의미를 갖고있지 않은 일련번호를 제거합니다.
del titanic_df['Ticket'] # ticket 번호에서 패턴이 확인되지 않음으로 제거합니다.


titanic_df['Title'] = titanic_df['Name'].str.extract('([A-Za-z]+)\.', expand=False) # 정규식을 사용하여 필요한 단어만 추출합니다.
del titanic_df['Name']

rare_title = [] # 호칭중에 갯수가 10개를 넘지 않는 호칭은 rare_title 빈 리스트 안에 넣어줍니다.
for title in set(titanic_df['Title']):
    if list(titanic_df['Title']).count(title) < 10:
        rare_title.append(title)

# 알 수 없는 호칭들은 알아볼 수 있게 바꿔줍니다.
titanic_df['Title'] = titanic_df['Title'].replace('Mlle', 'Miss') # Mademoiselle
titanic_df['Title'] = titanic_df['Title'].replace('Ms', 'Miss') 
titanic_df['Title'] = titanic_df['Title'].replace('Mme', 'Mrs') # Madame
titanic_df['Title'] = titanic_df['Title'].replace(rare_title, 'Rare')

# 딥러닝에서는 string 값은 읽을 수 없으니 모두 int로 만들어 줍니다.
title_mapping = {"Master":1, "Miss":2, "Mr":3, "Mrs":4, "Rare":5 }
titanic_df['Title'] = titanic_df['Title'].map(title_mapping)
titanic_df['Title'] = titanic_df['Title'].fillna(0)
titanic_df['Title'].astype(int)

# 성별도 int형으로 바꿔줍니다.
sex_mapping = {"male": 0 , "female":1} 
titanic_df['Sex'] = titanic_df['Sex'].map(sex_mapping)

# 승선 열에서 빈 곳을 'S'로 채워줍니다.
titanic_df['Embarked'] = titanic_df['Embarked'].fillna('S')

# 승선 열도 string형을 int 형으로 모두 mapping 시켜줍니다.
mapping_data ={"S":0, "Q":1, "C":2}
titanic_df["Embarked"] = titanic_df["Embarked"].map(mapping_data) 

# 운임 열도 딥러닝의 성능을 높이기 위해 category를 나눠줍니다.
titanic_df.loc[ titanic_df['Fare'] <= 102, 'Fare'] = 0,
titanic_df.loc[(titanic_df['Fare'] > 102) & (titanic_df['Fare'] <= 204), 'Fare'] = 1,
titanic_df.loc[(titanic_df['Fare'] > 204) & (titanic_df['Fare'] <= 307), 'Fare'] = 2,
titanic_df.loc[ titanic_df['Fare'] > 307, 'Fare'] = 4

# 'Sibsp'와 'Parch'를 'FamilySize'로 합치고 남은 열은 제거해줍니다.
titanic_df["FamilySize"] = titanic_df["SibSp"] + titanic_df["Parch"] +1
del titanic_df['SibSp']
del titanic_df['Parch']

# 혼자인 가구수를 알기 위한 열
titanic_df['isAlone'] = 0
titanic_df.loc[titanic_df['FamilySize'] == 1, 'isAlone'] = 1

# FamilySize 1 ~ 9를 재정렬합니다.
family_mapping = {1: 0, 2: 0.4, 3: 0.8, 4: 1.2, 5: 1.6, 6: 2, 7: 2.4, 8: 2.8, 9: 3.2, 10: 3.6, 11: 4}
titanic_df['FamilySize'] = titanic_df['FamilySize'].map(family_mapping)

# Age의 N/A를 Age 열의 중앙값으로 채워줍니다.
titanic_df["Age"].fillna(titanic_df.groupby("Title")["Age"].transform("median"), inplace=True)

# Age의 구간을 16씩 잘라서 나눠줍니다.
titanic_df.loc[ titanic_df['Age'] <= 16, 'Age'] = 0,
titanic_df.loc[(titanic_df['Age'] > 16) & (titanic_df['Age'] <= 32), 'Age'] = 1,
titanic_df.loc[(titanic_df['Age'] > 32) & (titanic_df['Age'] <= 48), 'Age'] = 2,
titanic_df.loc[(titanic_df['Age'] > 48) & (titanic_df['Age'] <= 64), 'Age'] = 3,
titanic_df.loc[ titanic_df['Age'] > 64, 'Age'] = 4

# 데이터 전처리 된 데이터프레임을 출력합니다.
titanic_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,FamilySize,isAlone
0,0,3,0,1.0,0.0,0,3,0.4,0
1,1,1,1,2.0,0.0,2,4,0.4,0
2,1,3,1,1.0,0.0,0,2,0.0,1
3,1,1,1,2.0,0.0,0,4,0.4,0
4,0,3,0,2.0,0.0,0,3,0.0,1


### 2. Devide dataframe into X & Y

In [4]:
# 필요한 라이브러리를 임포트 합니다.
from sklearn import model_selection

In [5]:
# 뽑아내고 싶은 데이터 열을 따로 만듭니다.
titanic_target = titanic_df[['Survived']].copy()
titanic_data = titanic_df.copy()
del titanic_data['Survived']

In [6]:
# model_selection을 통해 데이터를 Train Data와 Test Data로 나눠줍니다.
train_data, test_data, train_label, test_label = model_selection.train_test_split(titanic_data, titanic_target, test_size=0.3, random_state=0)

print(train_data.shape)
print(test_data.shape)
print(train_label.shape)
print(test_label.shape)

(623, 8)
(268, 8)
(623, 1)
(268, 1)


In [7]:
test_label.head()
# index를 보면 잘 섞여진 것을 보실 수 있습니다.

,Survived
495,0
648,0
278,0
31,1
255,1


### 3. Change normal labels to one-hot labels

In [8]:
# 원하는 라이브러리를 임포트합니다.
from sklearn.preprocessing import OneHotEncoder

#### Onehotencoder란?
>단 하나의 값만 True이고 나머지는 모두 False인 인코딩을 말한다.
>즉, 1개만 Hot(True)이고 나머지는 Cold(False)이다.
>예를들면 [0, 0, 0, 0, 1]이다. 5번째(Zero-based 인덱스이므로 4)만 1이고 나머지는 0이다.

In [9]:
enc = OneHotEncoder(categories='auto')

In [10]:
# train_label을 OneHotEncoder에 fitting한다.
enc.fit(train_label)

OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [11]:
train_label = enc.transform(train_label).toarray()
train_label

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [12]:
# test_label을 OneHotEncoder에 fitting한다.
enc.fit(test_label)

OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [13]:
test_label = enc.transform(test_label).toarray()
test_label

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.

In [14]:
print(train_label.shape)
print(test_label.shape)

(623, 2)
(268, 2)


### 4. Build & Train the model

In [15]:
# 딥러닝을 사용하기 위해 tensorflow와 tensorflow의 layers를 임포트 해줍니다.
import tensorflow as tf
from tensorflow import layers

In [16]:
import os
tf.logging.set_verbosity(tf.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [17]:
# X_data의 열의 개수를 맞춰서 작성해야 한다.
X = tf.placeholder(tf.float32, [None, 8])
# Y_data의 열의 개수를 맞춰서 작성해야 한다.(One-Hot Vector)
Y = tf.placeholder(tf.float32, [None, 2])

In [18]:
# layers.dropout()은 True/False로 Training/Testing 여부를 결정해 줄 수 있습니다.
dropout_sign = tf.placeholder(tf.bool)

In [19]:
# units는 perceptron의 개수를 의미하며, activation을 ReLU를 쓸 경우 kernel_initializer를 he_normal()을 쓰는 것을 권장한다.
L1 = layers.dense(X, 256, activation=tf.nn.relu, kernel_initializer=tf.keras.initializers.he_normal())  
# layers.dropout()의 "rate"는 keeping rate가 아닌 dropping rate이며, default 값은 0.5입니다.
L1 = layers.dropout(L1, rate=0.2, training=dropout_sign)

In [20]:
L2 = layers.dense(L1, 256, activation=tf.nn.relu, kernel_initializer=tf.keras.initializers.he_normal()) 
L2 = layers.dropout(L2, rate=0.2, training=dropout_sign)

In [21]:
L3 = layers.dense(L2, 256, activation=tf.nn.relu, kernel_initializer=tf.keras.initializers.he_normal()) 
L3 = layers.dropout(L3, rate=0.2, training=dropout_sign) 

In [22]:
L4 = layers.dense(L3, 256, activation=tf.nn.relu, kernel_initializer=tf.keras.initializers.he_normal()) 
L4 = layers.dropout(L4, rate=0.2, training=dropout_sign) 

In [23]:
# 2 -> Y_data의 열의 개수를 맞춰야 합니다.
model = layers.dense(L4, 2, activation=None) # 2 == # of Y_data's columns

In [24]:
# Cost Function은 예측 값과 실제 값의 차이를 기반으로 모델의 성능(정확도)을 판단하기 위한 함수입니다.
# Linear Regression의 경우 Mean Squared Error Function(평균 제곱 오차 함수)을 활용합니다.
# 여기서는 Softmax-Cross_Entropy를 사용합니다.
cost = tf.losses.softmax_cross_entropy(Y, model)
# Cost Function이 최소가 되도록 하는 θ를 찾게하는 것이 AdamOptimizer입니다.
optimizer = tf.train.AdamOptimizer(1e-2).minimize(cost) # 1e-2 == 0.01

In [25]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [26]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [27]:
# epoch이 도는 것을 진행률을 시각화합니다.
from tqdm import trange, tqdm_notebook

In [28]:
total_epoch = 100
total_cost = 0 # cost

for epoch in tqdm_notebook(range(total_epoch)):
    train_cp = [] # Training accuracy 를 동시에 출력해보도록 합니다.
    
    # 매 Epoch마다 Total cost를 출력합니다.
    _, cost_val = sess.run([optimizer, cost], feed_dict={X: train_data, Y: train_label, dropout_sign: True}) # , dropout_sign: True
    total_cost += cost_val # cost

    # 매 Epoch마다 Training/Test accuracy를 출력합니다. (dropout_sign을 False로 바꾸어 dropout을 걷어내줘야 합니다.)
    train_cp += sess.run([is_correct], feed_dict={X: train_data, Y: train_label, dropout_sign: False}) # , dropout_sign: False
    test_accuracy = sess.run([accuracy], feed_dict={X: test_data, Y: test_label, dropout_sign: False})
    
    if epoch % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 
              '|| Avg. cost =', '{:.3f}'.format(total_cost / (epoch + 1)), # cost
              '|| Training accuracy : {:.3f}'.format(np.mean(train_cp)), # Training accuracy
              '|| Test accuracy : {:.3f}'.format(float(test_accuracy[0]))) # Test accuracy

print('Learning process is completed!')

Epoch: 0001 || Avg. cost = 1.790 || Training accuracy : 0.388 || Test accuracy : 0.373
Epoch: 0011 || Avg. cost = 6.710 || Training accuracy : 0.695 || Test accuracy : 0.709
Epoch: 0021 || Avg. cost = 3.797 || Training accuracy : 0.745 || Test accuracy : 0.757
Epoch: 0031 || Avg. cost = 2.734 || Training accuracy : 0.820 || Test accuracy : 0.813
Epoch: 0041 || Avg. cost = 2.179 || Training accuracy : 0.820 || Test accuracy : 0.817
Epoch: 0051 || Avg. cost = 1.837 || Training accuracy : 0.838 || Test accuracy : 0.828
Epoch: 0061 || Avg. cost = 1.605 || Training accuracy : 0.846 || Test accuracy : 0.825
Epoch: 0071 || Avg. cost = 1.436 || Training accuracy : 0.843 || Test accuracy : 0.836
Epoch: 0081 || Avg. cost = 1.308 || Training accuracy : 0.848 || Test accuracy : 0.828
Epoch: 0091 || Avg. cost = 1.208 || Training accuracy : 0.848 || Test accuracy : 0.825

Learning process is completed!


In [29]:
# Test accuracy를 출력합니다. dropout_sign을 False로 바꾸어 dropout을 걷어내줘야 합니다.
print('Test accuracy : {}'.format(sess.run(accuracy, feed_dict={X: test_data, Y: test_label, dropout_sign: False})))

Test accuracy : 0.8246268630027771


In [30]:
# 모델이 실제로 예측한 값을 출력합니다.
predicted_labels = sess.run(tf.argmax(model, 1), feed_dict={X: test_data, dropout_sign: False})
print(predicted_labels)

[0 0 0 1 1 0 1 1 1 1 0 1 0 1 1 1 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 0
 0 1 0 0 0 0 0 1 1 0 0 1 1 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0
 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1
 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0
 0 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1
 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0]


In [31]:
# 연산에 쓰인 시스템 자원을 돌려보내려면 session을 닫아야 합니다.
sess.close()